# Part 12: Train an Encrypted NN on Encrypted Data

In this notebook, we're going to use all the techniques we've learned thus far to perform neural network training (and prediction) while both the model and the data are encrypted.

Note that Autograd is not *yet* supported for encrypted variables, thus we'll have to roll our own gradients ourselves. This functionality will be added in the next PySyft version.

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Jason Paumier - Github: [@Jasopaum](https://github.com/Jasopaum)

# Step 1: Create Workers and Toy Data

In [1]:
import torch
import syft as sy

In [2]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [3]:
# Create our dataset

data = torch.Tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
target = torch.Tensor([[0.], [0.], [1.], [1.]])

In [4]:
# Create our model
# We use a small linear model: M.X = y
# where M is the mdoel, X is the data, and y the prediction

model = torch.Tensor([[0.], [0.]])
learning_rate = 0.1

# Step 2: Encrypt the Model and Data

Encryption here comes in two steps. Since Secure Multi-Party Computation only works on Longs, in order to operate over numbers with decimal points (such as weights and activations), we need to encode all of our numbers using Fixed Precision, which will give us several bits of floating point precision. We do this by calling .fix_precision().

We can then call .share() as we have for other demos, which will encrypt all of the values by sharing them between Alice and Bob.

In [5]:
# We encode everything
prec_frac = 2

data_enc = data.fix_precision(precision_fractional=prec_frac).share(alice, bob, crypto_provider=james)
target_enc = target.fix_precision(precision_fractional=prec_frac).share(alice, bob, crypto_provider=james)
model_enc = model.fix_precision(precision_fractional=prec_frac).share(alice, bob, crypto_provider=james)

# The learning rate is also changed to fixed precision
learning_rate_fp = torch.tensor(learning_rate).fix_precision(precision_fractional=prec_frac)

# Step 3: Train

And now we can train using simple tensor logic. Note that autograd is not yet supported (but it will be in the Torch 1.0 refactor which you can [watch here](https://github.com/OpenMined/PySyft/issues/1587)).

In [6]:
for i in range(20):
    # Use the model to predict
    pred = data_enc.matmul(model_enc)
    # How much is the model wrong
    err = pred - target_enc
    # Compute gradient descent step
    update = data_enc.t().matmul(err)
    # Apply the gradient descent step
    model_enc = model_enc - update * learning_rate_fp
    
    loss = err.get().abs().sum().float_precision()
    print(loss) 

===
{'alice': (Wrapper)>[PointerTensor | me:22406171387 -> alice:28323756150], 'bob': (Wrapper)>[PointerTensor | me:97718244538 -> bob:35886260787]}
tensor([10])
___
tensor([2.])
===
{'alice': (Wrapper)>[PointerTensor | me:67500623698 -> alice:81757808369], 'bob': (Wrapper)>[PointerTensor | me:29593827223 -> bob:16479130358]}
tensor([10])
___
tensor([1.6000])
===
{'alice': (Wrapper)>[PointerTensor | me:63735358614 -> alice:1028675112], 'bob': (Wrapper)>[PointerTensor | me:98735404593 -> bob:90770182299]}
tensor([10])
___
tensor([1.3000])
===
{'alice': (Wrapper)>[PointerTensor | me:34752760639 -> alice:60234838606], 'bob': (Wrapper)>[PointerTensor | me:50272365042 -> bob:26772812228]}
tensor([10])
___
tensor([1.0600])
===
{'alice': (Wrapper)>[PointerTensor | me:75918384151 -> alice:5512535435], 'bob': (Wrapper)>[PointerTensor | me:97354276596 -> bob:94605523603]}
tensor([10])
___
tensor([0.8800])
===
{'alice': (Wrapper)>[PointerTensor | me:59971563483 -> alice:52881272976], 'bob': (Wrap

The loss indeed decreased.

We can also verify that the model is close to what we could have expected (the task was to output the value of the first dimension of the input).

In [7]:
model_enc.get().float_precision()

tensor([[0.9400],
        [0.0600]])

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on Github

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft Github Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for github issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)